<a href="https://colab.research.google.com/github/jvrscak/UUIProjekt/blob/main/drugi_zadatak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DRUGI ZADATAK - simulacija prometa

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import random